<a href="https://colab.research.google.com/github/ejihoon6065/-/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [38]:
!ls

'5.4.Sentiment Analysis.ipynb'	 log	   preprocessing  'Untitled Folder'
'5.4.Sentiment Analysis.zip'	 log.zip   test.tsv
 data				 model	   train.tsv


In [25]:
cd 5.4.Sentiment Analysis

[Errno 20] Not a directory: '5.4.Sentiment Analysis.ipynb'
/content/Sentiment Analysis


In [21]:
pwd

'/content/Sentiment Analysis'

In [30]:
import pandas as pd
from model.LSTM import LSTM_model
from preprocessing.text_preprocessing import Sentences2LemmaWords
from preprocessing.text_preprocessing import GetUnique
from preprocessing.text_preprocessing import setTokenizer
from preprocessing.text_preprocessing import LemmaWords2Seqeunces
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import time
from log.logger import Logger

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [40]:
folder_path = './data/'
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\t')

print('train data shape: ', train.shape)
print('test data shape: ', test.shape)
print('----------[train infomation]----------')
print(train.info())
print('----------[test infomation]----------')
print(test.info())

train data shape:  (156060, 4)
test data shape:  (66292, 3)
----------[train infomation]----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB
None
----------[test infomation]----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66292 entries, 0 to 66291
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PhraseId    66292 non-null  int64 
 1   SentenceId  66292 non-null  int64 
 2   Phrase      66292 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.5+ MB
None


In [41]:
train_tokens = Sentences2LemmaWords(train.Phrase)
test_tokens = Sentences2LemmaWords(test.Phrase)

2020-07-14 08:30:56,306[DEBUG|preprocessing, 17] Sentence2LemmaWords: start
/content/Sentiment Analysis/preprocessing/text_preprocessing.py:23: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 23 of the file /content/Sentiment Analysis/preprocessing/text_preprocessing.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  sentence = BeautifulSoup(sentence).get_text()


280 / 156060 Phrase

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


10901 / 156060 Phrase

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'log'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


...

155982 / 156060 Phrase

2020-07-14 08:33:03,566[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 127.26092290878296sec
2020-07-14 08:33:03,567[DEBUG|preprocessing, 17] Sentence2LemmaWords: start


156060 / 156060 Phrase
66064 / 66292 Phrase

2020-07-14 08:33:57,687[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 54.11944055557251sec


66292 / 66292 Phrase


In [42]:
unique_words, maxlen = GetUnique(train_tokens)
print(len(unique_words), maxlen)

2020-07-14 08:33:57,701[DEBUG|preprocessing, 44] GetUnique: start
2020-07-14 08:33:57,834[DEBUG|preprocessing, 56] GetUnique: finish. 계산 시간 0.13320517539978027sec


13745 48


In [43]:
target = train.Sentiment
y_target = to_categorical(target)
num_classes = y_target.shape[1]

In [44]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_tokens,
    y_target,
    test_size=0.2,
    stratify=y_target)

In [45]:
tokenizer = setTokenizer(x_train, len(unique_words))
x_train = LemmaWords2Seqeunces(x_train, tokenizer, maxlen)
x_valid = LemmaWords2Seqeunces(x_valid, tokenizer, maxlen)
x_test = LemmaWords2Seqeunces(test_tokens, tokenizer, maxlen)

2020-07-14 08:33:59,099[DEBUG|preprocessing, 63] setTokenizer: start
2020-07-14 08:34:00,172[DEBUG|preprocessing, 70] setTokenizer: finish. 계산 시간 1.0727298259735107sec
2020-07-14 08:34:00,174[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 08:34:01,823[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 1.6493120193481445sec
2020-07-14 08:34:01,828[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 08:34:02,177[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 0.3494713306427002sec
2020-07-14 08:34:02,180[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 08:34:02,789[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 0.6090283393859863sec


In [46]:
model = LSTM_model(len(unique_words), maxlen, num_classes)

logger = Logger('main')
start_time = time.time()
logger.debug('fit start')

model.fit(x_train,
          y_train,
          validation_data=(x_valid, y_valid),
          epochs=5,
          batch_size=1024,
          verbose=1)

fit_time = time.time() - start_time
logger.debug('fit end. fit_time {}'.format(fit_time))

2020-07-14 08:34:02,806[DEBUG|LSTM_model_generate, 9] LSTM_model: start
2020-07-14 08:34:10,343[DEBUG|LSTM_model_generate, 20] LSTM_model: finish
2020-07-14 08:34:10,345[DEBUG|main, 5] fit start
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 124848 samples, validate on 31212 samples
Epoch 1/5
124848/124848 [==============================] - 24s 196us/step - loss: 1.1018 - accuracy: 0.5628 - val_loss: 0.9091 - val_accuracy: 0.6281
Epoch 2/5
124848/124848 [==============================] - 22s 177us/step - loss: 0.8132 - accuracy: 0.6686 - val_loss: 0.8395 - val_accuracy: 0.6558
Epoch 3/5
124848/124848 [==============================] - 22s 177us/step - loss: 0.7337 - accuracy: 0.6985 - val_loss: 0.8307 - val_accuracy: 0.6621
Epoch 4/5
124848/124848 [==============================] - 22s 180us/step - loss: 0.6881 - accuracy: 0.7152 - val_loss: 0.8338 - val_accuracy: 0.6640
Epoch 5/5
124848/124848 [==============================] - 22s 179us/step - loss: 0.6543 - accuracy: 0.7258 - val_loss: 0.8435 - val_accuracy: 0.6618


2020-07-14 08:36:04,805[DEBUG|main, 15] fit end. fit_time 114.45991396903992


In [47]:
y_pred = model.predict_classes(x_test)

submission = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': y_pred
})
submission.to_csv(folder_path + 'submission.csv', index=False)